# data process

## 一、 user_pay
	1.1 数据读取
	1.2 数据统计
	1.3 数据平滑
	1.4 特征工程
## 二、 节假日特征
## 三、天气特征
## 四、 shop info
    4.1 数据读取
    4.2 one-hot
## 五、 user_view
    5.1 数据读取
    5.2 数据统计
    5.3 特征工程
## 六、 构造线下测试集方式


### 背景介绍
从2015.07.01到2016.10.31（除去2015.12.12）的商家数据，用户支付行为数据以及用户浏览行为数据。

预测测试集中所有商家在未来14天（2016.11.01-2016.11.14）


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

## 一、 user_pay
### 1.1 读取数据

In [2]:

# 过滤支付数据
user_pay = pd.read_csv('data/dataset/user_pay.txt', names=['user_id','shop_id','time_stamp'] )
print (user_pay.shape, len(set(user_pay['shop_id'])))


(69674110, 3) 2000


In [3]:
user_pay['dt'] = user_pay['time_stamp'].apply(lambda x:x[:10])
user_pay.head()

,user_id,shop_id,time_stamp,dt
0,22127870,1862,2015-12-25 17:00:00,2015-12-25
1,3434231,1862,2016-10-05 11:00:00,2016-10-05
2,16955285,1862,2016-02-10 15:00:00,2016-02-10
3,13799128,1862,2016-01-13 14:00:00,2016-01-13
4,13799128,1862,2016-07-05 12:00:00,2016-07-05


### 1.2 统计所有店铺在每一天的支付次数

In [5]:
count_user_pay = user_pay.groupby(['shop_id','dt'])['user_id'].count()
count_user_pay

shop_id  dt        
1        2015-10-10    188
         2015-10-11    272
         2015-10-12    257
         2015-10-13    239
         2015-10-14    277
                      ... 
2000     2016-10-27    132
         2016-10-28    133
         2016-10-29     67
         2016-10-30     67
         2016-10-31    168
Name: user_id, Length: 599275, dtype: int64

### 1.3 数据平滑

如果这一天的数据为0，则将该天用前三个星期对应天的平均值替换
从最后last_days开始计算，这里用后123天


In [6]:

import datetime

def get_date_list(start, end):
    date_list = []
    date = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    while date <= end:
        date_list.append(date.strftime('%Y-%m-%d'))
        date = date + datetime.timedelta(1)
    return date_list

dates = get_date_list('2015-07-01','2016-10-31')


In [1]:

infos_init = []
for shop_id in range(1,2001):
    for dt in dates:
        infos_init.append({
            'shop_id':shop_id,
            'dt':dt
        })
infos_init = pd.DataFrame(infos_init)

infos = []
for index,freq in tqdm(zip(count_user_pay.index, count_user_pay)):
    shop_id, dt = index
    infos.append({
            'shop_id':shop_id,
            'dt':dt,
            'pay_time':freq
        })

infos = pd.DataFrame(infos)


NameError: name 'dates' is not defined

In [8]:
count_user_pay = pd.merge(infos_init,infos,how='left',on=['shop_id', 'dt']).fillna(0)
count_user_pay.head()

,shop_id,dt,pay_time
0,1,2015-07-01,0.0
1,1,2015-07-02,0.0
2,1,2015-07-03,0.0
3,1,2015-07-04,0.0
4,1,2015-07-05,0.0


In [9]:
count_user_pay[count_user_pay['pay_time']!=0]

,shop_id,dt,pay_time
101,1,2015-10-10,188.0
102,1,2015-10-11,272.0
103,1,2015-10-12,257.0
104,1,2015-10-13,239.0
105,1,2015-10-14,277.0
...,...,...,...
977995,2000,2016-10-27,132.0
977996,2000,2016-10-28,133.0
977997,2000,2016-10-29,67.0
977998,2000,2016-10-30,67.0


In [10]:
# 历史靠近2015-07-01日期的样本无需统计，不在建模范畴内
infos = []

# for shop_id in tqdm(set(count_user_pay['shop_id'])):

    dta = count_user_pay[count_user_pay['shop_id']==shop_id].sort_values('dt',ascending=False).reset_index(drop=True)
    for i in range(len(dta)-30):
        if dta['pay_time'].values[i]==0:
            pt = (dta['pay_time'].values[i+7] + dta['pay_time'].values[i+14] + dta['pay_time'].values[i+21])/3
        else:
            pt = dta['pay_time'].values[i]
        infos.append({
            'shop_id':shop_id,
            'dt':dta['dt'].values[i],
            'pay_time':pt
        })
        

100%|██████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:23<00:00, 86.82it/s]


In [11]:
count_user_pay = pd.DataFrame(infos)

### 1.4 特征工程

In [12]:

def feature_pay_time_rate( df, feature_days ):
    '''
        # 变化率特征
    '''
    amounts = list(df['pay_time'])
    trade_date = list(df['dt'])
    infos = []
    for offset in range(len(amounts)-feature_days+1):
        info = {
                'dt':trade_date[offset],
                'pay_time':amounts[offset]
                }
        for i in range(feature_days-1):
            #与最近日期进行对比 -- 支付次数变化率
            if amounts[offset]!=0:
                info['pay_time'+'_day%s'%str(i+1)] = amounts[offset+i+1]/amounts[offset]
            else:
                info['pay_time'+'_day%s'%str(i+1)] = -1
            if i>0:
                if amounts[offset]!=0:
                    # 累计最近i天支付次数总和，相对于当日变化率
                    info['pay_time'+'_sum_day_%s'%str(i+1)] = sum(amounts[offset:offset+i+1])/amounts[offset]/(i+1)
                else:
                    info['pay_time'+'_sum_day_%s'%str(i+1)] = -1
        infos.append(info)
    
    return pd.DataFrame(infos)

shop = count_user_pay[count_user_pay['shop_id']==28].sort_values('dt',ascending=False)
ptr = feature_pay_time_rate( shop, feature_days=7 )
shop.head()


,shop_id,dt,pay_time
12393,28,2016-10-31,53.0
12394,28,2016-10-30,99.0
12395,28,2016-10-29,84.0
12396,28,2016-10-28,79.0
12397,28,2016-10-27,69.0


In [13]:
ptr.head()

,dt,pay_time,pay_time_day1,pay_time_day2,pay_time_sum_day_2,pay_time_day3,pay_time_sum_day_3,pay_time_day4,pay_time_sum_day_4,pay_time_day5,pay_time_sum_day_5,pay_time_day6,pay_time_sum_day_6
0,2016-10-31,53.0,1.867925,1.584906,1.433962,1.490566,1.484277,1.301887,1.485849,1.264151,1.449057,1.207547,1.418239
1,2016-10-30,99.0,0.848485,0.797980,0.924242,0.696970,0.882155,0.676768,0.835859,0.646465,0.804040,0.676768,0.777778
2,2016-10-29,84.0,0.940476,0.821429,0.970238,0.797619,0.920635,0.761905,0.889881,0.797619,0.864286,1.226190,0.853175
3,2016-10-28,79.0,0.873418,0.848101,0.936709,0.810127,0.907173,0.848101,0.882911,1.303797,0.875949,1.227848,0.947257
4,2016-10-27,69.0,0.971014,0.927536,0.985507,0.971014,0.966184,1.492754,0.967391,1.405797,1.072464,1.173913,1.128019


In [14]:

import numpy as np

def feature_pay_time_ma(df):
    '''
        # 构造3日、5日、10日 均线特征 + 排序
    '''
    ma7 = []
    ma14 = []
    ma28 = []
    for i in df.index[:-11]:
        ma7.append(np.mean(df['pay_time'].values[i:7+i]))
        ma14.append(np.mean(df['pay_time'].values[i:14+i]))
        ma28.append(np.mean(df['pay_time'].values[i:28+i]))
    
    infos = df[:-11].copy()
    infos['ma7'] = ma7
    infos['ma14'] = ma14
    infos['ma28'] = ma28
    
    maVl = infos[['ma7', 'ma14', 'ma28']]
    
    df = infos.copy()
    trade_date = df[['dt']]
    df = df[['ma7', 'ma14', 'ma28', 'pay_time']].copy()
    df = df.rename(columns={'pay_time':'rank_pay_time', 'ma7':'rank_ma7', 'ma14':'rank_ma14', 'ma28':'rank_ma28'})
    df = df.rank(axis=1)
    df['dt'] = trade_date
    
    df = pd.concat([df,maVl], axis=1)
    return df[['dt', 'ma7', 'ma14', 'ma28', 'rank_pay_time', 'rank_ma7', 'rank_ma14', 'rank_ma28']]

shop = count_user_pay[count_user_pay['shop_id']==28].sort_values('dt',ascending=False).reset_index(drop=True)
maDf = feature_pay_time_ma( shop )
maDf.head()


,dt,ma7,ma14,ma28,rank_pay_time,rank_ma7,rank_ma14,rank_ma28
0,2016-10-31,73.571429,72.357143,73.357143,1.0,4.0,2.0,3.0
1,2016-10-30,75.571429,72.357143,75.500000,4.0,3.0,1.0,2.0
2,2016-10-29,76.142857,71.428571,75.500000,4.0,3.0,1.0,2.0
3,2016-10-28,78.000000,71.428571,76.785714,4.0,3.0,1.0,2.0
4,2016-10-27,78.285714,71.500000,77.892857,1.0,4.0,2.0,3.0


## 二、 节假日特征

In [15]:

import requests

## api获取假期特征
def isHoliday(date):
    """
        2:假日
    
    """
    return requests.get('http://www.easybots.cn/api/holiday.php?d=%s'%date).content
holidayInfo = eval(isHoliday( get_date_list('2020-09-01','2020-12-01') ))
print (isHoliday('2016-10-01'))


b'{"20161001":"0"}'


## 三、 天气特征
    10霾~晴
    33雷阵雨~中到大雨
    02多云~小雨
    03晴~雷阵雨
    13阴~雷阵雨
    33暴雨转大雨
    33中雨~大到暴雨
    13雾~中雨
    02多云转小雨
    11阴~雾
    13阴转中雨
    33大到暴雨~暴雨
    33大雨转雷阵雨
    33中雨~雷阵雨
    33大暴雨~暴雨
    33大雨转暴雨
    33暴雨转中雨
    02多云~小雪

## 四、 shop info
对类别型变量进行one-hot处理

In [16]:
shop_info = pd.read_csv('data/dataset/shop_info.txt',
                names=['shop_id','city_name','location_id','per_pay','score','comment_cnt', 'shop_level','cate_1_name','cate_2_name','cate_3_name'])
shop_info.head()

,shop_id,city_name,location_id,per_pay,score,comment_cnt,shop_level,cate_1_name,cate_2_name,cate_3_name
0,1,湖州,885,8,4.0,12.0,2,美食,休闲茶饮,饮品/甜点
1,2,哈尔滨,64,19,NaN,NaN,1,超市便利店,超市,NaN
2,3,南昌,774,5,3.0,2.0,0,美食,休闲茶饮,奶茶
3,4,天津,380,18,NaN,NaN,1,超市便利店,超市,NaN
4,5,杭州,263,2,2.0,2.0,0,美食,休闲食品,生鲜水果


In [17]:
shop_info = shop_info.fillna(-1)

In [18]:
shop_info1 = pd.get_dummies(shop_info[['city_name','cate_1_name','cate_2_name','cate_3_name']])
shop_info1.head()

,city_name_三亚,city_name_三明,city_name_上海,city_name_上饶,city_name_东莞,city_name_东营,city_name_中山,city_name_丽水,city_name_乐山,city_name_佛山,...,cate_3_name_西北菜,cate_3_name_西式快餐,cate_3_name_西餐,cate_3_name_闽菜,cate_3_name_零食,cate_3_name_面包,cate_3_name_面点,cate_3_name_饮品/甜点,cate_3_name_香锅/烤鱼,cate_3_name_麻辣烫/串串香
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
shop_info2 = shop_info[['location_id', 'per_pay', 'score', 'comment_cnt', 'shop_level']]

In [20]:
featureShop = pd.concat([shop_info1,shop_info2], axis=1)

## 五、 user_view

### 5.1 数据读取

In [21]:

user_view = pd.read_csv('data/dataset/user_view.txt', names=['user_id','shop_id','time_stamp'] )
print (user_view.shape)


(5556715, 3)


In [22]:
user_view['dt'] = user_view['time_stamp'].apply(lambda x:x[:10])
user_view.head()

,user_id,shop_id,time_stamp,dt
0,13201967,1197,2016-10-21 18:00:00,2016-10-21
1,19461365,1197,2016-06-28 23:00:00,2016-06-28
2,15022321,1197,2016-07-16 19:00:00,2016-07-16
3,5440872,1197,2016-07-15 07:00:00,2016-07-15
4,12594529,1197,2016-08-07 16:00:00,2016-08-07


### 5.2 数据统计

In [23]:
count_user_view = user_view.groupby(['shop_id','dt'])['user_id'].count()
count_user_view

shop_id  dt        
1        2016-06-22    10
         2016-06-23    18
         2016-06-24     5
         2016-06-25    22
         2016-06-26    15
                       ..
2000     2016-10-27    16
         2016-10-28    12
         2016-10-29    11
         2016-10-30     3
         2016-10-31    15
Name: user_id, Length: 233494, dtype: int64

In [24]:

infos = []
for index,freq in tqdm(zip(count_user_view.index, count_user_view)):
    shop_id, dt = index
    infos.append({
            'shop_id':shop_id,
            'dt':dt,
            'view_time':freq
        })

count_user_view = pd.DataFrame(infos)

233494it [00:00, 912015.29it/s]


In [25]:
count_user_view

,shop_id,dt,view_time
0,1,2016-06-22,10
1,1,2016-06-23,18
2,1,2016-06-24,5
3,1,2016-06-25,22
4,1,2016-06-26,15
...,...,...,...
233489,2000,2016-10-27,16
233490,2000,2016-10-28,12
233491,2000,2016-10-29,11
233492,2000,2016-10-30,3


### 5.3 特征工程

In [26]:

import numpy as np

def feature_view_time_ma(df):
    '''
        # 构造3日、5日、10日 均线特征 + 排序
    '''
    ma7 = []
    ma14 = []
    ma28 = []
    for i in df.index[:-11]:
        ma7.append(np.mean(df['view_time'].values[i:7+i]))
        ma14.append(np.mean(df['view_time'].values[i:14+i]))
        ma28.append(np.mean(df['view_time'].values[i:28+i]))
    
    infos = df[:-11].copy()
    infos['ma7'] = ma7
    infos['ma14'] = ma14
    infos['ma28'] = ma28
    
    maVl = infos[['ma7', 'ma14', 'ma28']]
    
    df = infos.copy()
    trade_date = df[['dt']]
    df = df[['ma7', 'ma14', 'ma28', 'view_time']].copy()
    df = df.rename(columns={'view_time':'rank_view_time', 'ma7':'rank_ma7', 'ma14':'rank_ma14', 'ma28':'rank_ma28'})
    df = df.rank(axis=1)
    df['dt'] = trade_date
    
    df = pd.concat([df,maVl], axis=1)
    df = df[['dt', 'ma7', 'ma14', 'ma28', 'rank_view_time', 'rank_ma7', 'rank_ma14', 'rank_ma28']]
    
    return pd.DataFrame(df.values, columns=[i+'_view' if i!='dt' else i for i in df.columns])

shop = count_user_view[count_user_view['shop_id']==28].sort_values('dt',ascending=False).reset_index(drop=True)
maDf = feature_view_time_ma( shop )
maDf.head()


,dt,ma7_view,ma14_view,ma28_view,rank_view_time_view,rank_ma7_view,rank_ma14_view,rank_ma28_view
0,2016-10-31,22.7143,23.1429,28.3571,3,1,2,4
1,2016-10-30,23.5714,23.7857,28.8571,4,1,2,3
2,2016-10-29,21.2857,23.2857,28.6071,2,1,3,4
3,2016-10-28,21,23.7857,29.1071,3,1,2,4
4,2016-10-27,20.4286,23.7857,30.1429,1,2,3,4


## 六、 构造线下测试集方式
训练集：划窗构造训练样本。划窗的大小与预测时间窗口保持一致。  
线下测试集：10-25，10-26，10-27，10-28，10-29，10-30，10-31。

In [27]:
test = user_pay[user_pay['dt']>='2016-10-25']
test = test.groupby(['shop_id','dt'])['user_id'].count()
test

shop_id  dt        
1        2016-10-25    255
         2016-10-26    172
         2016-10-27    226
         2016-10-28    257
         2016-10-29    251
                      ... 
2000     2016-10-27    132
         2016-10-28    133
         2016-10-29     67
         2016-10-30     67
         2016-10-31    168
Name: user_id, Length: 13961, dtype: int64

In [28]:

infos = []
for index,freq in tqdm(zip(test.index, test)):
    shop_id, dt = index
    infos.append({
            'shop_id':shop_id,
            'dt':dt,
            'view_time':freq
        })

test = pd.DataFrame(infos)


13961it [00:00, 734824.29it/s]
